In [ ]:
import pandas as pd
import random
from sklearn.metrics import roc_auc_score, roc_curve

In [ ]:
from sklearn.pipeline import Pipeline
data = pd.read_csv('../data/onti_c.csv')
data.drop(['Time', 'Crystal Position'], axis=1, inplace=True)
# data.drop(['R FWHM_RHEED', 'X FWHM_RHEED', 'Y FWHM_RHEED'], axis=1, inplace=True)
data.tail()

In [ ]:
# data['Anomaly'] = data['Coolness_RHEED']
# data['Anomaly'] = (data['Anomaly'] - data['Anomaly'].rolling(50).mean()).apply(lambda x: (1 if abs(x) >= 5 else 0))

data['Anomaly'] = data['Coolness_RHEED'].rolling(50).mean()
data['Anomaly'] = ((data['Anomaly']- data['Anomaly'].shift(-435)) > 5)

temp = data[11000:len(data.index)-1000]
anom_indexes = temp[temp['Anomaly'] == 1].index
norm_indexes = temp[temp['Anomaly'] == 0].index

In [ ]:
import sklearn
import numpy as np

num_data = len(data.index)

train_split = 1
num_train = int(train_split * num_data)
num_test = num_data - num_train

train = data.drop('Anomaly', axis=1).values[0:num_train]
# test = data.drop('Anomaly', axis=1).values[num_train:]
scaler = sklearn.preprocessing.StandardScaler()
train = scaler.fit_transform(train)
# test = scaler.transform(test)

num_x_signals = train.shape[1]

def get_data_entry(length=1000):
    # Allocate a new array for the batch of input-signals.
    x_shape = (length, 10000, num_x_signals)
    x_batch = np.zeros(shape=x_shape, dtype=np.float32)

    # Allocate a new array for the batch of output-signals.
    y_shape = (length, 1, 1)
    y_batch = np.zeros(shape=y_shape, dtype=np.float32)

    # Fill the batch with random sequences of data.
    for i in range(length):
        # Get a random start-index.
        # This points somewhere into the training-data.
        idx = random.choice(anom_indexes) if i % 2 == 0 else random.choice(norm_indexes)
            
        # Copy the sequences of data starting at this index.
        x_batch[i] = train[idx - 10000:idx]
        y_mean = train[idx + 385:idx + 435][0].mean()
#         y_batch[i] = [1 if (abs(train[idx + 435][0] - y_mean) >= 5) else 0]
        y_batch[i] = [1 if i % 2 == 0 else 0]
        
    return (x_batch, y_batch)

def batch_generator(batch_size):
    while True:
        yield get_data_entry(length=batch_size)
        
generator = batch_generator(2)


In [ ]:
import joblib
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

k_range = list(range(1,27))

param_grid = dict(n_neighbors = k_range)

# tmp = get_data_entry(320)
# x_train = tmp[0]
# y_train = tmp[1][:, :, 0]

x_train, x_test, y_train, y_test = train_test_split(data.drop('Anomaly', axis=1).values, data['Anomaly'].values, test_size=0.1, random_state=17)
scaler = sklearn.preprocessing.StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# x_train = data.drop('Anomaly', axis=1).values
# y_train = data['Anomaly'].values

knn = KNeighborsClassifier()
grid = GridSearchCV(knn, param_grid, cv = 2, scoring = 'roc_auc', verbose=3)
grid.fit(x_train, y_train)

pred = grid.best_estimator_.predict(x_test)
roc_auc_score(y_test, pred)

# pipeline = Pipeline([
#     ('clf', knn)
# ])

joblib.dump(grid.best_estimator_, 'sub_knn/model.pkl', compress = 4)